In [1]:
import librosa
import numpy as np
import pandas as pd
import os
import soundfile as sf
import preprocessing
from collections import Counter
import matplotlib.pyplot as plt
import helper

ImportError: DLL load failed while importing _path: Impossibile trovare il modulo specificato.

# 1. Normalizzazione dei segnali digitali

Per lo studio dei segnali audio occore effettuare alcune modifiche. In particolare, una delle prime fasi del preprocessing, consiste nel ridimensionamento dei bit depth e nella conversione in monocanale. 

## 1.1 Bit Depth & Canali

Tutti i file audio son stati convertiti in file .wav

In [ ]:
file_path = 'C://Users//mario//OneDrive//Desktop//Dataset - Copia - no duplicati//Non-Target//Right whale//RightWhale.wav'

In [ ]:
def convert_to_wav(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)
        wav_file_path = os.path.splitext(file_path)[0] + '.wav'
        sf.write(wav_file_path, y, sr, subtype='PCM_16')
        return wav_file_path
    except Exception as e:
        print(f"Errore durante la conversione in WAV del file '{file_path}': {e}")
        return None

con librosa i file sono stati caricati in monocanale a 16 bit depth. Di seguito un esempio su un file audio

In [ ]:
y, sr = librosa.load(file_path, sr=None, mono=True)
y_16bit = np.int16(y * 32767)

spettrogramma del file audio

In [ ]:
helper.create_spectrogram(file_path, sr)

questi sono i risultati in seguito alla prima fase di preprocessing

In [ ]:
df = pd.read_csv("audio_features_dataset_prep.csv")

In [ ]:
helper.prep_plot_channel_distribution(df)


In [ ]:
helper.prep_plot_bit_depth_distribution(df)